# Projeto Marinha do Brasil

## Laboratório de Processamento de Sinais - UFRJ

### Autor: Vinícius dos Santos Mello <viniciusdsmello@poli.ufrj.br>

In [2]:
import os
import sys
import pickle
import numpy as np
import time

from keras.utils import np_utils
from keras.models import load_model

import sklearn.metrics
from sklearn.externals import joblib

from Functions import TrainParameters as trnparams
from Functions import TrainFunctions
from Functions.StackedAutoEncoders import StackedAutoEncoders

import multiprocessing

init_time = time.time()

m_time = time.time()
print 'Time to import all libraries: '+str(m_time-init_time)+' seconds'

analysis_name = 'StackedAutoEncoder'

# Enviroment variables
data_path = os.getenv('OUTPUTDATAPATH')
results_path = os.getenv('PACKAGE_NAME')

# paths to export results
base_results_path = '%s/%s'%(results_path,analysis_name)
pict_results_path = '%s/pictures_files'%(base_results_path)
files_results_path = '%s/output_files'%(base_results_path)

# For multiprocessing purpose
num_processes = multiprocessing.cpu_count()

# Read data
m_time = time.time()

# Database caracteristics
database = '4classes'
n_pts_fft = 1024
decimation_rate = 3
spectrum_bins_left = 400
development_flag = False
development_events = 400

# Check if LofarData has created...
if not os.path.exists('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                      (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left)):
    print 'No Files in %s/%s\n'%(data_path,database)
else:
    #Read lofar data
    [data,trgt,class_labels] = joblib.load('%s/%s/lofar_data_file_fft_%i_decimation_%i_spectrum_left_%i.jbl'%
                                           (data_path,database,n_pts_fft,decimation_rate,spectrum_bins_left))


    m_time = time.time()-m_time
    print 'Time to read data file: '+str(m_time)+' seconds'

    # correct format
    all_data = data
    all_trgt = trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

    # Process data
    # unbalanced data to balanced data with random data creation of small classes

    # Same number of events in each class
    qtd_events_biggest_class = 0
    biggest_class_label = ''

    for iclass, class_label in enumerate(class_labels):
        if sum(all_trgt==iclass) > qtd_events_biggest_class:
            qtd_events_biggest_class = sum(all_trgt==iclass)
            biggest_class_label = class_label
        print "Qtd event of %s is %i"%(class_label,sum(all_trgt==iclass))
    print "\nBiggest class is %s with %i events"%(biggest_class_label,qtd_events_biggest_class)

    balanced_data = {}
    balanced_trgt = {}

    from Functions import DataHandler as dh
    m_datahandler = dh.DataHandlerFunctions()

    for iclass, class_label in enumerate(class_labels):
        if development_flag:
            class_events = all_data[all_trgt==iclass,:]
            if len(balanced_data) == 0:
                balanced_data = class_events[0:development_events,:]
                balanced_trgt = (iclass)*np.ones(development_events)
            else:
                balanced_data = np.append(balanced_data,
                                          class_events[0:development_events,:],
                                          axis=0)
                balanced_trgt = np.append(balanced_trgt,(iclass)*np.ones(development_events))
        else:
            if len(balanced_data) == 0:
                class_events = all_data[all_trgt==iclass,:]
                balanced_data = m_datahandler.CreateEventsForClass(
                    class_events,qtd_events_biggest_class-(len(class_events)))
                balanced_trgt = (iclass)*np.ones(qtd_events_biggest_class)
            else:
                class_events = all_data[all_trgt==iclass,:]
                created_events = (m_datahandler.CreateEventsForClass(all_data[all_trgt==iclass,:],
                                                                     qtd_events_biggest_class-
                                                                     (len(class_events))))
                balanced_data = np.append(balanced_data,created_events,axis=0)
                balanced_trgt = np.append(balanced_trgt,
                                          (iclass)*np.ones(created_events.shape[0]),axis=0)

    all_data = balanced_data
    all_trgt = balanced_trgt

    # turn targets in sparse mode
    from keras.utils import np_utils
    trgt_sparse = np_utils.to_categorical(all_trgt.astype(int))

Time to import all libraries: 1.90734863281e-05 seconds
Time to read data file: 0.939898014069 seconds
Qtd event of 0 is 12939
Qtd event of 1 is 29352
Qtd event of 2 is 11510
Qtd event of 3 is 23760

Biggest class is 1 with 29352 events
DataHandler Class: CreateEventsForClass
Original Size: (12939, 400)
DataHandler Class: CreateEventsForClass
Original Size: (29352, 400)
DataHandler Class: CreateEventsForClass
Original Size: (11510, 400)
DataHandler Class: CreateEventsForClass
Original Size: (23760, 400)


# Carregar parâmetros de treinamento

In [4]:
# Load train parameters

analysis_str = 'StackedAutoEncoder'
model_prefix_str = 'RawData'

trn_params_folder='%s/%s/%s_trnparams.jbl'%(results_path,analysis_str,analysis_name)
if os.path.exists(trn_params_folder):
    os.remove(trn_params_folder)
if not os.path.exists(trn_params_folder):
    trn_params = trnparams.SAENoveltyDetectionTrnParams(n_inits=1,
                                                       hidden_activation='tanh', # others tanh, relu, sigmoid, linear 
                                                       output_activation='linear',
                                                       n_epochs=500,
                                                       patience=20,
                                                       batch_size=128,
                                                       verbose=False,
                                                       optmizerAlgorithm='Adam',
                                                       metrics=['mean_squared_error'],
                                                       loss='kullback_leibler_divergence')
    trn_params.save(trn_params_folder)
else:
    trn_params = trnparams.SAENoveltyDetectionTrnParams()
    trn_params.load(trn_params_folder)
    
# Choose how many fold to be used in Cross Validation
n_folds = 10
CVO = trnparams.NoveltyDetectionFolds(folder=results_path,n_folds=n_folds,trgt=all_trgt,dev=development_flag, verbose=True)
print '\n'+trn_params.get_params_str()

Reading from /home/vinicius.mello/Workspace/SonarAnalysis/Results/NoveltyDetection/10_folds_cross_validation.jbl

1_inits_mapstd_norm_500_epochs_128_batch_size_tanh_hidden_activation_linear_output_activation_mean_squared_error_metric_kullback_leibler_divergence_loss


# Treinamento do Stacked Auto Encoder

In [ ]:
##### Train Process #####
inovelty = 0
novelty_train=[inovelty]
print 'Novelty to Train: %i'%inovelty

SAE = StackedAutoEncoders(params           = trn_params,
                          development_flag = development_flag,
                          n_folds          = n_folds,
                          save_path        = results_path,
                          CVO              = CVO,
                          noveltyDetection = True,
                          inovelty         = inovelty)

# Choose layer to be trained
layer = 1

n_folds = len(CVO[inovelty])

hidden_neurons = range(400,0,-50) + [2]
print hidden_neurons

regularizer = ""
regularizer_param = 0.5

trn_data = all_data[all_trgt!=inovelty]
trn_trgt = all_trgt[all_trgt!=inovelty]
trn_trgt[trn_trgt>inovelty] = trn_trgt[trn_trgt>inovelty]-1

# Functions defined to be used by multiprocessing.Pool()
def trainNeuron(ineuron):
    for ifold in range(n_folds):
        SAE.trainLayer(data = trn_data,
                       trgt = trn_trgt,
                       ifold = ifold,
                       hidden_neurons = hidden_neurons + [ineuron],
                       layer = layer,
                       regularizer = regularizer,
                       regularizer_param = regularizer_param)

def trainFold(ifold):
    return SAE.trainLayer(data = trn_data,
                          trgt = trn_trgt,
                          ifold = ifold,
                          hidden_neurons = hidden_neurons,
                          layer = layer,
                          regularizer = regularizer,
                          regularizer_param = regularizer_param)

start_time = time.time()

# Start Parallel processing
p = multiprocessing.Pool(processes=num_processes)

####################### SAE LAYERS ############################
# It is necessary to choose the layer to be trained

# To train on multiple cores sweeping the number of neurons
folds = range(len(CVO[inovelty]))
results = p.map(trainFold, folds)

# To train multiple topologies sweeping the number of neurons
# neurons_mat = range(0,400,50) (start,final,step)
# results = p.map(trainNeuron, neurons_mat)

p.close()
p.join()

# for ifold in range(len(CVO[inovelty])):
#     result = trainFold(ifold)

end_time = time.time() - start_time
print "It took %.3f seconds to perform the training"%(end_time)

Novelty to Train: 0
[400, 350, 300, 250, 200, 150, 100, 50, 2]
Layer: 1 - Neuron: 400 - Fold 1 of 10 Folds -  Init 1 of 1 Inits
Layer: 1 - Neuron: 400 - Fold 7 of 10 Folds -  Init 1 of 1 Inits
Layer: 1 - Neuron: 400 - Fold 8 of 10 Folds -  Init 1 of 1 Inits
Layer: 1 - Neuron: 400 - Fold 6 of 10 Folds -  Init 1 of 1 Inits
Layer: 1 - Neuron: 400 - Fold 4 of 10 Folds -  Init 1 of 1 Inits


/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(

Layer: 1 - Neuron: 400 - Fold 2 of 10 Folds -  Init 1 of 1 Inits


/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))


Layer: 1 - Neuron: 400 - Fold 3 of 10 Folds -  Init 1 of 1 Inits


...

Layer: 1 - Neuron: 400 - Fold 5 of 10 Folds -  Init 1 of 1 Inits


/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:223: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:225: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform")`
  model.add(Dense(data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:252: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`

INFO (theano.gof.compilelock): Waiting for existing lock by process '25033' (I am process '25034')
INFO (theano.gof.compilelock): Waiting for existing lock by process '25033' (I am process '25031')
INFO (theano.gof.compilelock): Waiting for existing lock by process '25033' (I am process '25038')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/vinicius.mello/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.3.1611-Core-x86_64-2.7.5-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '25033' (I am process '25035')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/vinicius.mello/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.3.1611-Core-x86_64-2.7.5-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /home/vinicius.mello/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.3.1611-Core-x86_64-2.7.5-64/lock_dir
INFO (theano.gof.compilelock): To manual

/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:225: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform")`
  model.add(Dense(data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:252: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform", input_dim=400)`
  model.add(Dense(hidden_neurons[layer-1], input_dim=proj_all_data.shape[1], init="uniform"))
/home/vinicius.mello/Workspace/SonarAnalysis/Functions/StackedAutoEncoders.py:260: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, kernel_initializer="uniform")`
  model.add(Dense(proj_all_data.shape[1], init="uniform"))
/home/vinicius.mello/.virtualenvs/sonarenvSatigny/lib/python2.7/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
/home/

Layer: 1 - Neuron: 400 - Fold 9 of 10 Folds -  Init 1 of 1 Inits
Layer: 1 - Neuron: 400 - Fold 10 of 10 Folds -  Init 1 of 1 Inits


In [ ]:
results

In [ ]:
projection = SAE.getDataProjection(data = all_data, trgt=all_trgt, ifold = 0,
                                   hidden_neurons = hidden_neurons, layer = layer)
print projection.shape


# Treinamento de Classificação

In [ ]:
# Train classifiers to their corresponding folds
def trainClassifierFold(ifold): 
    return SAE.trainClassifier(data=all_data,
                        trgt = all_trgt,
                        ifold = 0,
                        hidden_neurons=hidden_neurons,
                        layer = 3)

# Train classifier sweeping the number of layer
def trainClassifierLayer(ilayer):
    for ifold in range(len(CVO)):
        SAE.trainClassifier(data=all_data,
                            trgt = all_trgt,
                            ifold = ifold,
                            hidden_neurons=hidden_neurons,
                            layer = ilayer)
start_time = time.time()

p = multiprocessing.Pool(num_processes)

folds = range(len(CVO))
results = p.map(trainClassifierFold, folds)

# layers = range(1,11)
# results = p.map(trainClassifierLayer, layers)

p.close()
p.join()

end_time = time.time() - start_time
print "It took %.3f seconds to perform the training"%(end_time)

In [ ]:
classifier = SAE.loadClassifier(data = all_data, trgt = all_trgt, hidden_neurons=hidden_neurons, layer=3, ifold = 0)

In [ ]:
train_id, test_id = CVO[0]
output = classifier.predict(all_data[test_id, :])
print sklearn.metrics.classification_report(trgt_sparse[test_id], np.round(output), target_names=class_labels.values())